### NIR spectroscopy and PCA

In [ ]:
# NIR data from Spectrum 100N
import os, sys
import pandas as pd
import cv2
import numpy as np
pd .options.display.max_rows = 10
df = pd.read_csv('Data_of_4_Meranti species.csv')

Firstly, plotting all the data to see their visual appearence.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

wavnum = np.flip(df.axes[1][1:])
wvnm= wavnum.astype(float)
sp_data=np.flip(df.values[:,1:],axis=1)
plt.figure(figsize=(12,6))
plt.plot(wvnm,sp_data[2:32].T)
plt.title("Whole spectra", fontsize=16)
plt.xlabel("wavenumber cm-1", fontsize=16)
plt.grid(which="both")
plt.gca().invert_xaxis()
plt.tick_params(labelsize=14) 

plt.show()

- Fig. Calculate 2nd derivative spectra by Savitzky-Golay algorithm. Window-size = 35, fitting function order = 3, seems reasonable. Window-size more than 41 seems to spoil the small peaks around 4000 cm-1. Spectral region 9000 - 10000 should be negrected.

In [ ]:
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    """
    .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
       Data by Simplified Least Squares Procedures. Analytical
       Chemistry, 1964, 36 (8), pp 1627-1639.
    .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
       W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
       Cambridge University Press ISBN-13: 9780521880688
    """
    import numpy as np
    from math import factorial
    
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError :
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

In [ ]:
plt.figure(figsize=(12,6))
y1 = sp_data[10,:]
y2 = savitzky_golay(y1,35,3,2)
plt.plot(wvnm,y2*500+0.5)
plt.plot(wvnm,y1.T+0.6)
plt.title("Whole and 2nd derivative spectra", fontsize=16)
plt.xlabel("wavenumber cm-1", fontsize=16)
plt.grid()
plt.tick_params(labelsize=14) 
plt.gca().invert_xaxis()

In [ ]:
df_2nd = []
for n in list(range(len(df.axes[0]))):
    x = df.values[n,1:]
    df_2nd.append(savitzky_golay(x,35,3,2))
X_data = np.flip(np.asarray(df_2nd),axis=1)
target_names=np.unique(df.values[:,0])
tmp = pd.get_dummies(df.values[:,0])
X_category = tmp.values.argmax(1) # make strings into numbers 0,1,2,...
sp_names = df.values[:,0]

In [ ]:
# PCA analyses : wavenuber range  8000-4000 cm-1
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler as SS
from mpl_toolkits.mplot3d import Axes3D

colors = ['r', 'g', 'b','y' ]

wavenumber_max=8000
wavenumber_min=4000
wv_max=int(np.where(wavnum==str(wavenumber_max))[0])
wv_min= int(np.where(wavnum==str(wavenumber_min))[0])

y1= X_data[:,wv_min:wv_max]
X = y1*10e3  # 
title = "PCA from "+str(wavenumber_max)+" to "+str(wavenumber_min)+ " region"
y = X_category
pca = PCA(n_components=10)
X_r = pca.fit(X).transform(X)
        

fig = plt.figure(figsize=(12,10),dpi=200)
plt.subplots_adjust(wspace=0.4, hspace=0.6)

ax1 = fig.add_subplot(5, 1, 1)  # overlap years and correlation coefficients

ax1.bar([1, 2,3,4,5,6,7,8,9,10], pca.explained_variance_ratio_, align = "center")
ax1.set_title("Screeplot of "+title)
ax1.set_xlabel("components")
ax1.set_ylabel("contribution")


ax1.legend(frameon=False, loc ='upper left')
ax0.legend(frameon=False)
ax0.set_title(ttl1) 

ax2 = fig.add_subplot(5, 1, 2) 
ax2.gca().invert_xaxis()
ax2.plot(wvnm[wv_min:wv_max],y1.T*10e3,linewidth = 2 ) ## x 10e3 
ax2.set_title("2nd dev spectra of "+title)

ax2 = fig.add_subplot(5, 1, 3) 
loadings = pca.components_*np.c_[np.sqrt(pca.explained_variance_)]
ax3.figure(figsize=(10,3))
ax3.gca().invert_xaxis()
ax3.plot(wvnm[wv_min:wv_max],loadings[0],linewidth = 2 )
plt.plot(wvnm[wv_min:wv_max],loadings[1],linewidth = 2 )
plt.plot(wvnm[wv_min:wv_max],loadings[2],linewidth = 2 )
plt.title("Loading of "+title)
plt.show()



ax2.plot(final_res['Year'],final_res['Master chronology'],'b-', label='master')  
ax2.plot(final_res['Year'][keyY:keyY+q],df['test1_L5MA'][0:q], 'g-', label='test')
ax2.set_ylabel('ring wiidth (log/5yrsAv)')
ax2.legend(frameon=False)
ax2.set_title(ttl2) 





# scree plot of PCA 



# Original spectrum



# PCA Loading

        
#PCA 3D plot
fig = plt.figure(1, figsize=(8, 6))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
plt.cla()
lw=5
for color, i, target_name in zip(colors, list(range(max(X_category)+1)), target_names):
    ax.scatter(X_r[y==i, 0], X_r[y==i, 1], X_r[y==i, 2], color=color, lw=lw, label=target_name) 
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title(title, fontsize=20)
ax.legend(bbox_to_anchor=(1.1, 0.8), shadow=False, scatterpoints=1)
         
# PCA
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,4))
lw = 2 # line width
for color, i, target_name in zip(colors, list(range(max(X_category)+1)), target_names):
    ax1.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color,lw=lw,label=target_name)
ax1.set_title(title)
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')

for color, i, target_name in zip(colors, list(range(max(X_category)+1)), target_names):
    ax2.scatter(X_r[y == i, 1], X_r[y == i, 2], color=color, lw=lw,label=target_name)
ax2.legend(bbox_to_anchor=(1.4, 1.1), shadow=False, scatterpoints=1)
ax2.set_title(title)
ax2.set_xlabel('PC2')
ax2.set_ylabel('PC3')
plt.show()


#### End of program